# Shubham Shrama
## IIT BOMBAY
In this repository,  we'll impliment U-Net network on the cell dataset.

In [0]:
#As my data is on my drive in Cell_data folder, we'll have to mount the drive
from google.colab import drive
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras.optimizers import RMSprop,SGD, adam
 
from keras import callbacks 
from keras.models import Model
from sklearn.model_selection import train_test_split

#from GeneralizationPOC.NetPixel import TransferLearning_19_Heart,TransferLearning_19_Test_rbc,U_Net
import numpy as np
import cv2

np.random.seed(1337) # for reproducibility
import keras

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
#from FirstPackage.Network import net_1
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.85
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:
width=512
height=512
depth=3
classes=2
weightsPath=None
keras.backend.set_image_data_format('channels_first')

In [0]:
def U_Net(width, height, depth, classes,weightsPath=None):
        
    bn_flag=True
    data_shape = height*width
    
    inputs = Input((depth,width, height))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1,training=bn_flag)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization(axis=1)(conv1,training=bn_flag)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization(axis=1)(conv2,training=bn_flag)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization(axis=1)(conv2,training=bn_flag)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization(axis=1)(conv3,training=bn_flag)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization(axis=1)(conv3,training=bn_flag)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization(axis=1)(conv4,training=bn_flag)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization(axis=1)(conv4,training=bn_flag)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    
    conv10 = Conv2D(classes, (3, 3), padding='same')(conv9)
             
    reshape = Reshape((classes, data_shape), input_shape=(classes, height, width))(conv10)
    perm = Permute((2, 1))(reshape)
    softmax = Activation("softmax")(perm)
    
    model = Model(inputs=[inputs], outputs=[softmax])
   
    if weightsPath is not None:
        model.load_weights(weightsPath)
    
    print(model.summary())
    return model 

  

In [0]:
pathtoload='gdrive/My Drive/Cell_data/'
#Loading training and testing sets
x_train=np.load(pathtoload+'x_train.npy')
y_train=np.load(pathtoload+'y_train.npy')
x_test=np.load(pathtoload+'x_test.npy')
y_test=np.load(pathtoload+'y_test.npy')
x_train_whitened=np.load(pathtoload+'x_train_whitened.npy')
x_test_whitened=np.load(pathtoload+'x_test_whitened.npy')


In [0]:
path2save = './Cell.h5'

#Define Image Size and Number of classes
imheight = 512
imwidth = 512
imdepth = 3
data_shape = imheight*imwidth
classes = 2

# #Load numpy arrays from the directory
# train_data = np.load(train_data_path+'data_cell.npy')
# # train_data = np.transpose(train_data,[0,2,3,1])
# #train_data = train_data.astype("float32")
# label = np.reshape(label,(len(label),data_shape,classes))
y_train = np.reshape(y_train,(len(y_train),data_shape,classes))
y_test = np.reshape(y_test,(len(y_test),data_shape,classes))




In [0]:
#Create Network Graph in the Memory
net = U_Net(imwidth, imheight, imdepth, classes,weightsPath='Cell.h5_0200-1.0000.h5')
#print(net.summary())

print ("Compiling Model...")
#Store the network weights whenever loss is minimum than previous epoch
modelCheck = callbacks.ModelCheckpoint(path2save+'_{epoch:04d}-{acc:.4f}.h5', monitor='acc', mode='auto')
# modelCheck = callbacks.ModelCheckpoint(path2save, monitor='acc', verbose=0, save_best_only=True, mode='auto')
 
opt = adam(lr=1e-4)
#Set the compiler parameter for the training
net.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"],sample_weight_mode='auto')
print ("Training the Model...")
 
#Train the Network
net.fit(x_train_whitened, y_train, batch_size = 18, epochs= 500, verbose=1,callbacks= [modelCheck],validation_data=(x_test_whitened, y_test))
print ("Dumping Weights to file...")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 3, 512, 512)  0                                            
__________________________________________________________________________________________________
conv2d_153 (Conv2D)             (None, 32, 512, 512) 896         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_65 (BatchNo (None, 32, 512, 512) 128         conv2d_153[0][0]                 
__________________________________________________________________________________________________
conv2d_154 (Conv2D)             (None, 32, 512, 512) 9248        batch_normalization_65[0][0]     
__________________________________________________________________________________________________
batch_norm

KeyboardInterrupt: ignored